<a href="https://colab.research.google.com/github/kardwalker/Malaria-model-/blob/main/Mal_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls '/content/drive/MyDrive/Mal_model.save'

ls: cannot access '/content/drive/MyDrive/Mal_model.save': No such file or directory


In [ ]:
import numpy as np
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense ,Conv2D , InputLayer , Flatten , BatchNormalization  , Activation , AveragePooling2D , MaxPooling2D   , Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.regularizers import L2
from tensorflow.keras.metrics import Accuracy , Precision , Recall
from tensorflow.keras.callbacks import EarlyStopping , LearningRateScheduler
import matplotlib.pyplot as plt


Dataset

In [ ]:
dataset , info = tfds.load("malaria" ,with_info = True , as_supervised = True , shuffle_files = True , split = ["train"])
def train_val_test_split(dataset , train_ratio , val_ratio, test_ratio ):
  size = len(dataset)
  train_data = dataset.take(int(train_ratio*size))
  val_data = dataset.skip(int(train_ratio*size)).take(int(val_ratio*size))
  test_data = dataset.skip(int(train_ratio*size) + int(val_ratio*size))
  return train_data , val_data , test_data

train_ratio = 0.7 ; val_ratio = 0.1 ; test_ratio = 0.2
train_data , val_data, test_data = train_val_test_split(dataset[0] ,train_ratio , val_ratio , test_ratio )

In [ ]:
info

tfds.core.DatasetInfo(
    name='malaria',
    full_name='malaria/1.0.0',
    description="""
    The Malaria dataset contains a total of 27,558 cell images with equal instances
    of parasitized and uninfected cells from the thin blood smear slide images of
    segmented cells.
    """,
    homepage='https://lhncbc.nlm.nih.gov/publication/pub9932',
    data_dir='/root/tensorflow_datasets/malaria/1.0.0',
    file_format=tfrecord,
    download_size=337.08 MiB,
    dataset_size=317.62 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=27558, num_shards=4>,
    },
    citation="""@article{rajaraman2018pre,
      title={Pre-trained convolutional neural networks as feature extractors toward
      improved malaria parasite detection in thin blood smear im

In [ ]:
def resizie_rescale(image, label):
  image = tf.image.resize(image, (224, 224))/255.0
  return image, label

train_dataset = train_data.map(resizie_rescale)
val_dataset = val_data.map(resizie_rescale)
test_dataset = test_data.map(resizie_rescale)

train_dataset = train_dataset.shuffle(buffer_size = 8 , reshuffle_each_iteration=True , ).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size = 8 , reshuffle_each_iteration=True , ).batch(32).prefetch(tf.data.AUTOTUNE)
print(train_dataset)
print(val_dataset)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [ ]:
# Model architecture
model = tf.keras.Sequential([
    Conv2D(6, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 6)       168       
                                                                 
 batch_normalization_4 (Bat  (None, 222, 222, 6)       24        
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 111, 111, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 16)      880       
                                                                 
 batch_normalization_5 (Bat  (None, 109, 109, 16)      64        
 chNormalization)                                                
                                                      

In [ ]:
metrics = [
            Accuracy(name='accuracy'), Precision(name='precision'), Recall(name='recall'), ]

In [ ]:
metrics = [
             Accuracy(name = "accuracy") ,Precision(name='precision'), Recall(name='recall'), ]

model.compile(optimizer= Adam(learning_rate = 0.01),
              loss = BinaryCrossentropy(),
              metrics = metrics)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose = 1, patience=5, start_from_epoch = 3,min_delta = 0.00001  , restore_best_weights=False)


In [ ]:
def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
lr_schedule = LearningRateScheduler(
    scheduler, verbose=1
)

In [ ]:

hist = model.fit(train_dataset,validation_data= val_dataset,epochs=20 ,callbacks = [ lr_schedule , early_stopping],verbose =1)


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/20
268/603 [============>.................] - ETA: 11:24 - loss: 0.4940 - accuracy: 0.0000e+00 - precision: 0.7665 - recall: 0.8218


KeyboardInterrupt



In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss performace')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

NameError: name 'hist' is not defined

In [ ]:
test_dataset = test_dataset.batch(1)

model.evaluate(test_dataset)

In [ ]:
from tensorflow.keras.preprocessing import image


In [ ]:
from flask import Flask, request, render_template

import numpy as np
import os

app = Flask(__name__)
#model = load_model("path/to/your/mal_model")  # Update the path

def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

@app.route("/", methods=["GET", "POST"])
def predict():
    if request.method == "POST":
        if 'file' not in request.files:
            return "No file part"
        file = request.files['file']
        if file.filename == '':
            return "No selected file"
        if file:
            file_path = os.path.join("uploads", file.filename)
            file.save(file_path)
            img_array = prepare_image(file_path)
            prediction = model.predict(img_array)
            if prediction[0][0] > 0.5:
                result = "Infected with Malaria"
            else:
                result = "Not Infected with Malaria"
            return result
    return '''
    <!doctype html>
    <title>Upload an Image</title>
    <h1>Upload an Image of Blood Cell</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload>
    </form>
    '''

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
